In [19]:
#!pip install gradio
#!pip install pyttsx3
#!pip install transformers
#!pip install num2words
#!pip install faster_whisper
!pip install accelerate


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import re
from scipy.io import wavfile
import os
import gradio as gr
import numpy as np
import pyttsx3
from transformers import pipeline
import torch
import gc
import random
import re
from num2words import num2words
from faster_whisper import WhisperModel
torch.cuda.empty_cache()
gc.collect()
# apt-get install espeak
# apt-get install ffmpeg
HUGGINGFACE_TOKEN = "Your Token"


def convert_numbers_to_words(text):
    def replace_number(match):
        num = int(match.group())
        return num2words(num, to='currency')
    return re.sub(r'\b\d+\b', replace_number, text)


def split_text_with_pauses(text):
    text = ''.join(i for i in text if not i.isdigit())
    chunks = re.split(r'[^\w\s]', text)
    
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    
    result = []
    for chunk in chunks:
        result.append(chunk)
        result.append("pause")
    
    return result


def chunk_into_segments(str_lst, words_per_segment=5):
    adjusted_str_lst = []
    for text in str_lst:
        if len(text.split(' ')) <= words_per_segment:
            adjusted_str_lst.append(text)
        else:
            tmp_triplet = ""
            triplet_counter = 0
            word_lst = text.split(' ')
            totals_words = len(word_lst)
            for word_idx_start in range(0, totals_words, words_per_segment):
                word_idx_end = word_idx_start+words_per_segment
                if word_idx_end < totals_words:
                    segment = " ".join(word_lst[word_idx_start:word_idx_end])
                else:
                    segment = " ".join(word_lst[word_idx_start:])
                adjusted_str_lst.append(segment)
    return adjusted_str_lst


def setup_engine(language="english_wmids"):
    # language = "default"  # english_wmids, english_rp"  # "english"
    engine = pyttsx3.init()  # "sapi5")
    voices = engine.getProperty("voices")
    available_voices = [voice_name.id for voice_name in engine.getProperty("voices")]
    selected_voice = available_voices[available_voices.index(language)]
    engine.setProperty('voice', selected_voice)
    # engine.setProperty('voice', selected_voice)
    engine.setProperty('rate', 120)
    engine.setProperty('volume', 0.8)
    engine.setProperty('pitch', 130)
    return engine


def text_to_speech(history):  # english_wmids"):
    if history == []:
        return "", tuple([22050, np.array([0])])
    text_input = history[-1][1]
    print("######################################################################")
    print(f">>> Voicing '{text_input}'...")
    print("######################################################################")
    language="english"
    if os.path.exists('final_audio.wav'):
        os.remove('final_audio.wav')
    get_text_to_speech_model = setup_engine(language)
    text_with_numbers_to_str = convert_numbers_to_words(text_input)
    text_with_pauses = split_text_with_pauses(text_with_numbers_to_str)
    text_with_pauses_adjusted = chunk_into_segments(text_with_pauses)
    print(f">>> Post processed to: {text_with_pauses_adjusted}")
    final_audio = []
    for text_segment in text_with_pauses_adjusted:
        print(text_segment)
        if text_segment=="pause":
            get_text_to_speech_model.save_to_file('.', 'tmp.wav')
        else:
            get_text_to_speech_model.save_to_file(text_segment, 'tmp.wav')
        get_text_to_speech_model.runAndWait() # Do not forget to add this line
        sr, data = wavfile.read('tmp.wav')
        final_audio.append(data)
        try:
            os.remove("tmp.wav")
        except:
            pass
    final_audio = np.concatenate(final_audio).astype(np.int16)
    # Save it back if needed
    # wav.write("final_audio.wav", sr, final_audio.astype(np.int16))
    print(final_audio.shape, final_audio.shape)
    return text_input, tuple([22050, final_audio])  # numpy_to_mp3(final_audio, 100)  # final_audio

    
def generate_response(user_input, audio_input, history):
    if audio_input:
        segments, _ = speech_to_text.transcribe(audio_input)
        text_input = ' '.join([segment.text for segment in segments])
    else:
        text_input = user_input
    if text_input == "":
        response = random.choice([["What? I did not get that!"], ["Come again?"], ["Huh?"], ["What?"]])
        history.append((text_input, response))
        return history, response, None, "", None
    print("######################################################################")
    print(f">>> Generating response to: {text_input}")
    print("######################################################################")
    messages = [
        # {"role": "system", "content": "Talk with user naturally"},
        {"role": "user", "content": text_input},
        {"role": "system", "content": "Make sure to talk with short, funny, sarcastic, and cynical tone"}
        # {"role": "system", "content": "I always reply to user with short, funny, sarcastic, and cynical tone"}
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.2)
    generated_text = outputs[0]["generated_text"]
    response = generated_text[len(prompt):].strip()  # Remove the prompt part
    response = ' '.join(generated_text.split('<|assistant|>')[1:])
    if history is None:
        history = []
    history.append((text_input, response))
    return history, response, None, "", None


speech_to_text = WhisperModel("small", device="cpu", compute_type="int8")

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                torch_dtype=torch.float16, device_map="auto",
                token=HUGGINGFACE_TOKEN)
    
def startup_llm_bot():
    
    def terminate():
        block.close()  # Close the Gradio application window
        exit()

    with gr.Blocks() as block:
        gr.HTML(
            f"""
            <h1 style='text-align: center;'> Rusty - The Smart Bot 🤖 </h1>
            <h3 style='text-align: center;'> Ask a question and receive the answer you will regret getting...</h3>
            """
        )

        chat_history = gr.Chatbot(label="Chat History")  # Chat history window

        with gr.Group():
            with gr.Row():
                audio_out = gr.Audio(label="Spoken Answer", streaming=True, autoplay=True)
                answer = gr.Textbox(label="Answer")
                state = gr.State()
            with gr.Row():
                # User input textbox for typing a question
                user_input = gr.Textbox(label="Message (press Enter to send):", placeholder="Type your question here...", elem_id="user_input")
                # User input for audio question
                audio_in = gr.Audio(label="Speak (press again when done):", sources="microphone", type="filepath", elem_id="audio_in")
                # Button to send the input when pressed
            with gr.Row():
                gr.Markdown("")
            with gr.Accordion("💀 Terminate Bot 💀", open=False):
                # submit_button = gr.Button("Ask", elem_id="submit_button")
                terminate_button = gr.Button("💀", elem_id="terminate_button")  # Terminate button


        # Set up Textbox to submit on Enter press
        user_input.submit(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
            .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
            .then(lambda hist: hist, inputs=state, outputs=chat_history)
        # Set up Audio input to submit when recording ends or Enter key is pressed
        audio_in.stop_recording(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
            .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
            .then(lambda hist: hist, inputs=state, outputs=chat_history)
        # Set up submit button to trigger the function on button press
        # audio_in.stop_recording
        # submit_button.click(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
        #     .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
        #     .then(lambda hist: hist, inputs=state, outputs=chat_history)\
        #     .then()

        # audio_in.stop_recording(generate_response, inputs=[user_input, audio_in, state], outputs=[state, answer, audio_out, user_input, audio_in])\
        #     .then(fn=text_to_speech, inputs=state, outputs=[answer, audio_out])\
        #     .then(lambda hist: hist, inputs=state, outputs=chat_history)\
        #     .then()
        # Set up Terminate button to close the application
        terminate_button.click(terminate)  # Call the terminate function to stop Gradio app


    block.css = """
        #user_input, #audio_in {
            height: 200px;  /* Set height for user input, audio input, and button (button excluded) */
        }
        .gradio-container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .gradio-row {
            width: 100%;
            justify-content: center;
            align-items: center;
            margin-bottom: 15px;
        }
    """

    block.launch(server_name="0.0.0.0", share=True)

    
startup_llm_bot()

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_427/2104872087.py:162: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_history = gr.Chatbot(label="Chat History")  # Chat history window


* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://9be0cc00ecfdbfca98.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
